# Import librarires

In [1]:
import tkinter as tk
import pandas as pd
from pulp import *
from tkinter import *
import numpy as np

# Write the Function of optimization

<div style="direction:rtl">
    <strong>
    این تابع بهینه سازی برنامه رژیم را انجام می دهد. میزان دریافتی های کالری و سایر مواد غذایی برای هر وزن و جنسیت متفاوت است. همچنین رژیم بصورت پروتئینی است یعنی از مواد غذایی که حاوی پروتئین بالا و چرپی پایین تر است بیشتر مصرف می کند. این قید بصورت قید بهینه سازی ذکر شده است و همچنین برای هر مواد غذایی یک دسته بندی ابتدا چیده شده است و پس از آن نیز برای هر دسته بندی قید شده است که بیشتر از 3 مورد مصرف نشود و این باعث مشود تنوع سبد غذایی بیشتر شود. لازم به ذکر است که تابع بهینه سازی بر اساس بیشینه سازی مصرف مواد غذایی چیده شده است
    </strong>


 </div>

In [4]:
def mainfunc(gender, weight, percentage):
    weight = float(weight)
    if gender.lower() == 'man':
        Min = [1500,30,20,800,130,125,60,1000,400,700,10]
        Min = [x*weight*percentage/75 for x in Min]
        Max = [2500,240,70,2000,450,250,100,10000,5000,1500,40]
        Max = [x*percentage*weight/75 for x in Max]
    else:
        Min = [1500,30,20,800,130,125,60,1000,400,700,10]
        Min = [x*percentage*weight / (75*2) for x in Min]
        Max = [2500,240,70,2000,450,250,100,10000,5000,1500,40]
        Max = [x*percentage*weight / (75*2) for x in Max]
    df = pd.read_csv("USDA.csv")
    df = df.fillna(df.mean())
    categories = []
    for i in df.Description.values:
        category = re.findall(r'\s|,|[^,\s]+', i)[0]
        categories.append(category)
    df['Cat'] = categories
    df['Description'] = df['Description'].replace('/',' ')

    df_sorted = df.sort_values(by=['Protein','TotalFat'], ascending=[True, False])
    protein_choices = df_sorted[df_sorted.Protein>50]['ID'].values

    uniq_cat = np.unique(df['Cat'])
    food_items = list(df['ID'])
    calories = dict(zip(food_items,df['Calories']))
    cholesterol = dict(zip(food_items,df['Cholesterol']))
    fat = dict(zip(food_items,df['TotalFat']))
    sodium = dict(zip(food_items,df['Sodium']))
    carbs = dict(zip(food_items,df['Carbohydrate']))
    fiber = dict(zip(food_items,df['Sugar']))
    protein = dict(zip(food_items,df['Protein']))
    vit_E = dict(zip(food_items,df['VitaminE']))
    vit_C = dict(zip(food_items,df['VitaminC']))
    vit_D = dict(zip(food_items,df['VitaminD']))
    calcium = dict(zip(food_items,df['Calcium']))
    iron = dict(zip(food_items,df['Iron']))
    pots = dict(zip(food_items,df['Potassium']))

    food_vars = LpVariable.dicts("Portion",food_items,0,cat='Continuous')

    food_chosen = LpVariable.dicts("Chosen",food_items,0,1,cat='Integer')

    prob = LpProblem("Simple Diet Problem",LpMaximize)

    prob += lpSum([calories[f] * food_vars[f] for f in food_items]) >= Min[0], "CalorieMinimum"
    prob += lpSum([calories[f] * food_vars[f] for f in food_items]) <= Max[0], "CalorieMaximum"

    # Cholesterol
    prob += lpSum([cholesterol[f] * food_vars[f] for f in food_items]) >= Min[1], "CholesterolMinimum"
    prob += lpSum([cholesterol[f] * food_vars[f] for f in food_items]) <= Max[1], "CholesterolMaximum"

    # Fat
    prob += lpSum([fat[f] * food_vars[f] for f in food_items]) >= Min[2], "FatMinimum"
    prob += lpSum([fat[f] * food_vars[f] for f in food_items]) <= Max[2], "FatMaximum"

    # Sodium
    prob += lpSum([sodium[f] * food_vars[f] for f in food_items]) >= Min[3], "SodiumMinimum"
    prob += lpSum([sodium[f] * food_vars[f] for f in food_items]) <= Max[3], "SodiumMaximum"

    # Carbs
    prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= Min[4], "CarbsMinimum"
    prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= Max[4], "CarbsMaximum"

    # Fiber
    prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) >= Min[5], "FiberMinimum"
    prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) <= Max[5], "FiberMaximum"

    # Protein
    prob += lpSum([protein[f] * food_vars[f] for f in food_items]) >= Min[6], "ProteinMinimum"
    prob += lpSum([protein[f] * food_vars[f] for f in food_items]) <= Max[6], "ProteinMaximum"

    # Vitamin A
    prob += lpSum([vit_D[f] * food_vars[f] for f in food_items]) >= Min[7], "VitaminAMinimum"
    prob += lpSum([vit_D[f] * food_vars[f] for f in food_items]) <= Max[7], "VitaminAMaximum"

    # Vitamin C
    prob += lpSum([vit_C[f] * food_vars[f] for f in food_items]) >= Min[8], "VitaminCMinimum"
    prob += lpSum([vit_C[f] * food_vars[f] for f in food_items]) <= Max[8], "VitaminCMaximum"

    # Calcium
    prob += lpSum([calcium[f] * food_vars[f] for f in food_items]) >= Min[9], "CalciumMinimum"
    prob += lpSum([calcium[f] * food_vars[f] for f in food_items]) <= Max[9], "CalciumMaximum"

    # Iron
    prob += lpSum([iron[f] * food_vars[f] for f in food_items]) >= Min[10], "IronMinimum"
    prob += lpSum([iron[f] * food_vars[f] for f in food_items]) <= Max[10], "IronMaximum"





    for f in food_items:
        prob += food_vars[f]>= food_chosen[f]*0.1
        prob += food_vars[f]<= food_chosen[f]*1e5

    prob += lpSum([food_chosen[i] for i in food_items])


    #Selective
    prob += lpSum([food_chosen[f] for f in food_items]) <= max(int(20*percentage),5), "MaxMeals"
    prob += lpSum([food_chosen[f] for f in food_items]) >= 2, "MinMeals"

    # for cat in uniq_cat:
    #     prob += lpSum([food_chosen[f] for f in df[df['Cat']==cat]['ID'].values.tolist()]) <= 5
    min_protein = 2
    if percentage == 0.15:
        min_protein = 1
    prob += lpSum([food_chosen[p] for p in protein_choices]) >= min_protein
    # prob += lpSum([food_chosen[p] for p in protein_choices]) <= 5

    prob.solve()
    return(prob,df,calories,protein)

# Graphical window for export diet plan

<div style="direction:rtl">
    <strong>
        در این قسمت ابتدا توابع مربوط به پنجره گرافیکی مینویسیم. در این توابع ابتدا ورودی ها از کاربر گرفته می شوند و سپس تابع مربوط به بهینه سازی ران می شود و در انتها نیز خروجی برنامه در پنجره گرافیکی نوشته می شود.
    </strong>
</div>

In [5]:
fields = 'Gender', 'Weight'

def fetch(entries):
    gender = entries[0][1].get()
    weight = entries[1][1].get()
    percentage_meals = {'Breakfast': 0.25,'Morning Snack': 0.15, 'Lunch': 0.35, 'Dinner': 0.25}
    txt = "The optimal (least cost) balanced diet with additional constraints \
            (e.g. at least 3 types of protein sources, consists of\n"+"-"*110+"\n"
    for meal, percentage in percentage_meals.items():
        prob,df,calories,protein = mainfunc(gender,weight,percentage)
        txt = txt + meal + " : \n"
        for v in prob.variables():
            if v.varValue>0 and v.name[0]=='P':
                tmp = v.name.replace('Portion_','')
                txt = txt + str(df[df.ID==int(tmp)]['Description'].values[0]) + "----in Category: " +  str(df[df.ID==int(tmp)]['Cat'].values[0]) + "----=" + str(v.varValue) + " Calories: " + str(round(v.varValue*calories[int(tmp)],2)) + " Protein: " + str(round(v.varValue*protein[int(tmp)],2)) + "\n"
    sample = Label(text=txt , anchor="e", justify=LEFT)
    sample.pack()

def makeform(root, fields):
    entries = []
    for field in fields:
        row = tk.Frame(root)
        lab = tk.Label(row, width=100, text=field, anchor='w')
        ent = tk.Entry(row)
        row.pack(side=tk.TOP, fill=tk.X, padx=5, pady=5)
        lab.pack(side=tk.LEFT)
        ent.pack(side=tk.RIGHT, expand=tk.YES, fill=tk.X)
        entries.append((field, ent))
    return entries

if __name__ == '__main__':
    root = tk.Tk()
    ents = makeform(root, fields)
    root.bind('<Return>', (lambda event, e=ents: fetch(e)))   
    b1 = tk.Button(root, text='Show',
                  command=(lambda e=ents: fetch(e)))
    b1.pack(side=tk.LEFT, padx=5, pady=5)
    b2 = tk.Button(root, text='Quit', command=root.quit)
    b2.pack(side=tk.LEFT, padx=5, pady=5)
    root.mainloop()

C:\Users\mamng\AppData\Local\Temp\ipykernel_7372\253250459.py:14: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = df.fillna(df.mean())
C:\Users\mamng\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
C:\Users\mamng\AppData\Local\Temp\ipykernel_7372\253250459.py:14: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = df.fillna(df.mean())
C:\Users\mamng\anaconda3\lib\site-packages\pulp\pulp.py: